In [ ]:
# Checking the contents in FileStore, the location where we uploaded our AWS credentials
dbutils.fs.ls("/FileStore/tables")

Out[221]: [FileInfo(path='dbfs:/FileStore/tables/authentication_credentials.csv', name='authentication_credentials.csv', size=202, modificationTime=1687110999000)]

In [ ]:
from pyspark.sql.functions import *
import urllib

In [ ]:
# Specify file type to be csv
file_type = "csv"
# Indicates file has first row as the header
first_row_is_header = "true"
# Indicates file has comma as the delimeter
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/authentication_credentials.csv")

In [ ]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='databricks-user').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [ ]:
AWS_S3_BUCKET = "user-0e4c2ab6fb3b-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/pdp_mount"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[225]: True

In [ ]:
# Check if the S3 bucket was mounted succesfully
display(dbutils.fs.ls("/mnt/pdp_mount/../..")) 

In [ ]:
# Specify the whole path to check the contents of a given topic in the S3 bucket
display(dbutils.fs.ls("/mnt/pdp_mount/topics/0e4c2ab6fb3b.user/partition=0/"))

In [ ]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
def read_json(record_type):
    '''
    Reads all the json files into a Spark dataframe 
    
    Parameters
    ----------
    record_type: str
        pin, geo or user 
    
    Returns
    -------
    df:
    resulting dataframe that shows all the json data read from the mounted S3 bucket
    '''
    file_location = f"/mnt/pdp_mount/topics/0e4c2ab6fb3b.{record_type}/partition=0/*.json" 
    file_type = "json"
    # Ask Spark to infer the schema
    infer_schema = "true"
    # Read in JSONs from mounted S3 bucket
    df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .load(file_location)
    return df

df_pin = read_json("pin")
df_geo = read_json("geo")
df_user = read_json("user")

In [ ]:
# Show Spark dataframe to check its content
df_pin.show(10)

+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
 category| description|downloaded|follower_count| image_src|index|is_image_or_video| poster_name| save_location| tag_list| title| unique_id|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
event-planning|Το όνομα που επέλ...| 1| 4|https://i.pinimg....| 4387| image|Manosbojana Katsa...|Local save in /da...|Diy Flowers,Flowe...| Βάπτιση: H παραμυ...|ae5e7377-f1bd-4ac...|
 home-decor|Традиционные швед...| 1| 136k|https://i.pinimg....| 6717| image|PUFIK Interiors &...|Local save in /da...|Scandinavian Cott...|〚 Уютные шведские...|bc5ab9ee-505e-44f...|
 home-decor|Традиционные швед...| 1| 136k|https://i.pinimg....| 6717| image|PUFIK Interiors &...|Local save in /da...|Scandinavian Cott...|〚 Уютные шведские...|bc5ab9ee-505e-44f...|
 home-decor|Традиционные швед...| 1| 136k|https://i.pinimg....| 6717| image|PUFIK Interiors &...|Local save in /da...|Scandinavian Cott...|〚 Уютные шведские...|bc5ab9ee-505e-44f...|
event-planning|15.1k Likes, 83 C...| 1| 311|https://i.pinimg....| 4858| image| Marie Bradford|Local save in /da...|Diy Birthday Deco...| THE EVENT COLLECT...|58101415-9273-431...|
event-planning|Wow your guests! ...| 1| 1k|https://i.pinimg....| 4608| image|Iconica Design | ...|Local save in /da...|Christmas Party B...| Virtual Baby Show...|d234e56f-5b18-4ef...|
 home-decor|6,636 Likes, 141 ...| 1| 394|https://i.pinimg....| 6633| image| Sarah Martin|Local save in /da...|Diy Kitchen Cabin...| The Cottage Journ...|d136f6bc-840d-44f...|
 home-decor|6,636 Likes, 141 ...| 1| 394|https://i.pinimg....| 6633| image| Sarah Martin|Local save in /da...|Diy Kitchen Cabin...| The Cottage Journ...|d136f6bc-840d-44f...|
 christmas|Features: Materia...| 1| 5k|https://i.pinimg....| 1706| image| Wear24-7|Local save in /da...|Merry Christmas T...| Standing Figurine...|b5c8a1b5-9e90-452...|
 christmas|Features: Materia...| 1| 5k|https://i.pinimg....| 1706| image| Wear24-7|Local save in /da...|Merry Christmas T...| Standing Figurine...|b5c8a1b5-9e90-452...|
+--------------+--------------------+----------+--------------+--------------------+-----+-----------------+--------------------+--------------------+--------------------+---------------------+--------------------+
only showing top 10 rows

In [ ]:
# Show Spark dataframe to check its content
df_geo.show(10)

+--------------------+-----+--------+---------+-------------------+
 country| ind|latitude|longitude| timestamp|
+--------------------+-----+--------+---------+-------------------+
British Indian Oc...| 9455|-82.9272| -150.346|2022-03-15 01:46:32|
British Indian Oc...| 6814|-86.5675| -149.565|2022-09-02 11:34:28|
British Indian Oc...| 6814|-86.5675| -149.565|2022-09-02 11:34:28|
British Indian Oc...| 5111|-83.7472| 8.65953|2021-04-01 00:56:57|
British Indian Oc...| 2989| -87.013| 133.062|2020-01-09 19:18:54|
Antarctica (the t...|10073|-32.8885| -170.295|2021-06-29 19:56:04|
Antarctica (the t...|10073|-32.8885| -170.295|2021-06-29 19:56:04|
Antarctica (the t...| 2418|-88.4642| -171.061|2022-05-27 11:30:59|
Antarctica (the t...| 2418|-88.4642| -171.061|2022-05-27 11:30:59|
Antarctica (the t...| 5162|-71.6607| -149.206|2019-09-27 19:06:43|
+--------------------+-----+--------+---------+-------------------+
only showing top 10 rows

In [ ]:
# Show Spark dataframe to check its content
df_user.show(10)

+---+-------------------+-----------+-----+---------+
age| date_joined| first_name| ind|last_name|
+---+-------------------+-----------+-----+---------+
 42|2017-02-18 00:31:22|Christopher| 6353|Hernandez|
 27|2016-03-08 13:38:37|Christopher| 2015| Bradshaw|
 59|2017-05-12 21:22:17| Alexander|10673|Cervantes|
 48|2016-02-27 16:57:44|Christopher| 1857| Hamilton|
 45|2016-09-15 06:02:53|Christopher|10020| Hawkins|
 27|2016-03-08 13:38:37|Christopher| 2015| Bradshaw|
 59|2017-05-12 21:22:17| Alexander|10673|Cervantes|
 27|2016-03-08 13:38:37|Christopher| 2015| Bradshaw|
 59|2017-05-12 21:22:17| Alexander|10673|Cervantes|
 48|2016-02-27 16:57:44|Christopher| 1857| Hamilton|
+---+-------------------+-----------+-----+---------+
only showing top 10 rows

1) Clean the DataFrame that contains information about Pinterest posts

In [ ]:
def clean_df_pin(df):
    '''
    Parameters
    ----------
    df:
        Dataframe containing relevant information
    
    Returns
    -------
    df_cleaned:
        Cleaned version of dataframe  
    '''
    # Replace empty entries and entries with no relevant data in each column with Nones
    df = df.select([when(col(c) == "", None).otherwise(col(c)).alias(c) for c in df.columns])

    df = df.withColumn("follower_count", when(df.follower_count.contains("User Info Error"), 0).otherwise(df.follower_count))

    df = df.withColumn("poster_name", when(df.poster_name.contains("User Info Error"), None).otherwise(df.poster_name))

    df = df.select([when(col(c).contains("N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e"), None).otherwise(col(c)).alias(c) for c in df.columns])

    df = df.select([when(col(c).contains("Image src error"), None).otherwise(col(c)).alias(c) for c in df.columns])

    df = df.select([when(col(c).contains("No description"), None).otherwise(col(c)).alias(c) for c in df.columns])

    df = df.select([when(col(c).contains("Untitled"), None).otherwise(col(c)).alias(c) for c in df.columns])

    df = df.select([when(col(c).contains("No Title Data Available"), None).otherwise(col(c)).alias(c) for c in df.columns])

    # Ensure that each column containing numeric data has a numeric data type
    df = df.withColumn('follower_count', when(df.follower_count.endswith('k'), regexp_replace(df.follower_count, 'k', '000')) \
        .when(df.follower_count.endswith('M'), regexp_replace(df.follower_count, 'M', '000000')) \
        .otherwise(df.follower_count))

    # change the datatype of the "follower_count" column to int
    df = df.withColumn("follower_count", df.follower_count.cast('int'))

    # Clean the data in the save_location column to include only the save location path
    df = df.withColumn('save_location', when(df.save_location.startswith('Local save in '), regexp_replace(df.save_location, 'Local save in ', '')))

    # Rename the index column to ind
    df = df.withColumnRenamed('index', 'ind')

    # Reorder the DataFrame columns
    df_cleaned = df.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")

    return df_cleaned

df_pin_cleaned = clean_df_pin(df_pin)
df_pin_cleaned.show(10)

+----+--------------------+---------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+----+--------------------+---------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
4387|ae5e7377-f1bd-4ac...| Βάπτιση: H παραμυ...|Το όνομα που επέλ...| 4|Manosbojana Katsa...|Diy Flowers,Flowe...| image|https://i.pinimg....|/data/event-planning|event-planning|
6717|bc5ab9ee-505e-44f...|〚 Уютные шведские...|Традиционные швед...| 136000|PUFIK Interiors &...|Scandinavian Cott...| image|https://i.pinimg....| /data/home-decor| home-decor|
6717|bc5ab9ee-505e-44f...|〚 Уютные шведские...|Традиционные швед...| 136000|PUFIK Interiors &...|Scandinavian Cott...| image|https://i.pinimg....| /data/home-decor| home-decor|
6717|bc5ab9ee-505e-44f...|〚 Уютные шведские...|Традиционные швед...| 136000|PUFIK Interiors &...|Scandinavian Cott...| image|https://i.pinimg....| /data/home-decor| home-decor|
4858|58101415-9273-431...| THE EVENT COLLECT...|15.1k Likes, 83 C...| 311| Marie Bradford|Diy Birthday Deco...| image|https://i.pinimg....|/data/event-planning|event-planning|
4608|d234e56f-5b18-4ef...| Virtual Baby Show...|Wow your guests! ...| 1000|Iconica Design | ...|Christmas Party B...| image|https://i.pinimg....|/data/event-planning|event-planning|
6633|d136f6bc-840d-44f...| The Cottage Journ...|6,636 Likes, 141 ...| 394| Sarah Martin|Diy Kitchen Cabin...| image|https://i.pinimg....| /data/home-decor| home-decor|
6633|d136f6bc-840d-44f...| The Cottage Journ...|6,636 Likes, 141 ...| 394| Sarah Martin|Diy Kitchen Cabin...| image|https://i.pinimg....| /data/home-decor| home-decor|
1706|b5c8a1b5-9e90-452...| Standing Figurine...|Features: Materia...| 5000| Wear24-7|Merry Christmas T...| image|https://i.pinimg....| /data/christmas| christmas|
1706|b5c8a1b5-9e90-452...| Standing Figurine...|Features: Materia...| 5000| Wear24-7|Merry Christmas T...| image|https://i.pinimg....| /data/christmas| christmas|
+----+--------------------+---------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
only showing top 10 rows

2) Clean the DataFrame that contains information about geolocation

In [ ]:
def clean_df_geo(df):
    '''
    Parameters
    ----------
    df:
        Dataframe containing relevant information
    
    Returns
    -------
    df_cleaned:
        Cleaned version of dataframe  
    '''
    # Create a new column coordinates that contains an array based on the latitude and longitude columns
    # Drop the latitude and longitude columns from the DataFrame
    # Reorder the DataFrame columns
    df = df.withColumn("coordinates", array("latitude", "longitude")) \
    .select("ind", "country", "coordinates", "timestamp")

    # Convert the timestamp column from a string to a timestamp data type
    df_cleaned = df.withColumn("timestamp", to_timestamp("timestamp"))

    return df_cleaned

df_geo_cleaned = clean_df_geo(df_geo)
df_geo_cleaned.show(10)

+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 9455|British Indian Oc...|[-82.9272, -150.346]|2022-03-15 01:46:32|
 6814|British Indian Oc...|[-86.5675, -149.565]|2022-09-02 11:34:28|
 6814|British Indian Oc...|[-86.5675, -149.565]|2022-09-02 11:34:28|
 5111|British Indian Oc...| [-83.7472, 8.65953]|2021-04-01 00:56:57|
 2989|British Indian Oc...| [-87.013, 133.062]|2020-01-09 19:18:54|
10073|Antarctica (the t...|[-32.8885, -170.295]|2021-06-29 19:56:04|
10073|Antarctica (the t...|[-32.8885, -170.295]|2021-06-29 19:56:04|
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
 5162|Antarctica (the t...|[-71.6607, -149.206]|2019-09-27 19:06:43|
+-----+--------------------+--------------------+-------------------+
only showing top 10 rows

3) Clean the DataFrame that contains information about users

In [ ]:
def clean_df_user(df):
    '''
    Parameters
    ----------
    df:
        Dataframe containing relevant information
    
    Returns
    -------
    df_user_cleaned:
        Cleaned version of dataframe  
    '''
    # Create a new column user_name that concatenates the information found in the first_name and last_name columns
    # Drop the first_name and last_name columns from the DataFrame by not selecting them
    df = df.withColumn("user_name", concat("first_name", "last_name")) \
        .select("age", "date_joined", "ind", "user_name")

    # Convert the date_joined column from a string to a timestamp data type
    # Reorder the DataFrame columns
    df_cleaned = df.withColumn("date_joined", to_timestamp("date_joined")) \
        .select("ind", "user_name", "age", "date_joined")
    
    return df_cleaned

df_user_cleaned = clean_df_user(df_user)
df_user_cleaned.show(10)

+-----+--------------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+--------------------+---+-------------------+
 6353|ChristopherHernandez| 42|2017-02-18 00:31:22|
 2015| ChristopherBradshaw| 27|2016-03-08 13:38:37|
10673| AlexanderCervantes| 59|2017-05-12 21:22:17|
 1857| ChristopherHamilton| 48|2016-02-27 16:57:44|
10020| ChristopherHawkins| 45|2016-09-15 06:02:53|
 2015| ChristopherBradshaw| 27|2016-03-08 13:38:37|
10673| AlexanderCervantes| 59|2017-05-12 21:22:17|
 2015| ChristopherBradshaw| 27|2016-03-08 13:38:37|
10673| AlexanderCervantes| 59|2017-05-12 21:22:17|
 1857| ChristopherHamilton| 48|2016-02-27 16:57:44|
+-----+--------------------+---+-------------------+
only showing top 10 rows

Creating functions to avoid repetition of code

In [ ]:
def join_df(df1, df2):
    '''
    Parameters
    ----------
    df1: 
        A dataframe to be joined
    df2:
        A dataframe to be joined
    
    Returns
    -------
    joined_df:
        A dataframe created by joining df1 and df2 on the ind column
    '''
    joined_df = df1.join(df2, on = "ind")
    return joined_df

def create_age_groups(df):
    '''
    Parameters
    ----------
    df:
        Dataframe that an age_group column will be added to

    Returns
    -------
    df: 
        Dataframe containing the newly added age_group column
    '''
    df = df.withColumn("age_group",
                       when(df.age.between(18, 24), "18-24")
                        .when(df_user_cleaned.age.between(25, 35), "25-35")
                        .when(df_user_cleaned.age.between(36, 50), "36-50")
                        .when(df_user_cleaned.age > 50, "50+"))
    return df

Registering the dataframes as TemporaryViews in order to run SQL queries against them

In [ ]:
df_pin_cleaned.createOrReplaceTempView("PIN")
df_geo_cleaned.createOrReplaceTempView("GEO")
df_user_cleaned.createOrReplaceTempView("USER")

4) Find the most popular category in each country

In [ ]:
df_most_popular_cat_by_country = join_df(df_pin_cleaned, df_geo_cleaned)
df_most_popular_cat_by_country = df_most_popular_cat_by_country.groupBy('country', 'category') \
        .agg(count('*').alias('count')) \
            .groupBy('country') \
                .agg(max(struct('count', 'category')).alias('max_count')) \
                    .select('country', 'max_count.category', 'max_count.count') \
                        .withColumnRenamed("count", "category_count")

df_most_popular_cat_by_country.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 60|
 Albania| mens-fashion| 91|
 Algeria| quotes| 115|
 American Samoa| education| 46|
 Andorra| tattoos| 34|
 Angola|diy-and-crafts| 36|
 Anguilla| home-decor| 38|
Antarctica (the t...| christmas| 35|
 Antigua and Barbuda| art| 46|
 Argentina| tattoos| 63|
 Armenia|diy-and-crafts| 85|
 Aruba| tattoos| 66|
 Australia| mens-fashion| 51|
 Austria| travel| 66|
 Azerbaijan| finance| 42|
 Bahamas| art| 19|
 Bahrain| finance| 19|
 Bangladesh| art| 39|
 Barbados| travel| 46|
 Belarus| travel| 5|
+--------------------+--------------+--------------+
only showing top 20 rows

Here, I've joined the pin and geolocation dataframes, calling the joined dataframe 'df_most_popular_cat_by_country'. I've then grouped the dataframe by the 'country' and 'category' columns, followed by adding an aggregate function that counts the total number of entries and calling it 'count'. Then, I've created a struct column from the 'category' and 'count' columns and applied the max() function to the struct column to return the maximum count for each category, calling the struct 'max_count'; then, I've grouped the dataframe by the country column. Finally, I've selected the 'country', 'max_country.category' and 'max_country.count' columns, renaming the final column as 'category_count'

5) Find which was the most popular category each year

In [ ]:
df_most_popular_category_by_year = join_df(df_pin_cleaned, df_geo_cleaned)
df_most_popular_category_by_year = df_most_popular_category_by_year.filter((year('timestamp') >= 2018) & (year('timestamp') <= 2022))
df_most_popular_category_by_year = df_most_popular_category_by_year.groupBy(year('timestamp').alias('post_year'), 'category') \
    .agg(count('*').alias('category_count'))

df_most_popular_category_by_year = df_most_popular_category_by_year.orderBy(['post_year', 'category_count'], ascending = [True, False]) \
    .groupBy('post_year') \
        .agg({'category': 'first', 'category_count': 'first'}) \
            .select('post_year', 'first(category)', 'first(category_count)') \
                .withColumnRenamed('first(category)', 'category') \
                    .withColumnRenamed('first(category_count)', 'category_count')

df_most_popular_category_by_year.show()

+---------+---------+--------------+
post_year| category|category_count|
+---------+---------+--------------+
 2018| travel| 178|
 2019|education| 167|
 2020| art| 187|
 2021| tattoos| 182|
 2022|christmas| 170|
+---------+---------+--------------+

Here, I've firstly joined the pin and geolocation dataframes, calling the resulting dataframe 'df_most_popular_category_by_year'. I've then filtered the dataframe to only return values where the year is between 2018 and 2022. I've then grouped the dataframe by the 'post_year' (renamed year('timestamp') column) and 'category' columns, adding a count() aggregate function that counts the total number of entries corresponding to each year and category respectively, naming it 'category_count'.

Next, I've ordered the dataframe by the 'post_name' and 'category_count' values, with the former appearing in asending order and the latter column appearing in descending order; I've then grouped the dataframe by the 'post_year' column and created another aggregate function whereby the first elements in both the 'category' and 'category_count' columns are selected. Finally, I selected the 'post_year' column as well as the first elements of both the 'category' and 'category_count' columns for each post year.

6) Find the user with most followers in each country

In [ ]:
from pyspark.sql.window import Window
df_most_followers_by_country = join_df(df_pin_cleaned, df_geo_cleaned)
df_most_followers_by_country = df_most_followers_by_country.groupBy('country', 'poster_name') \
        .agg(max('follower_count')) \
            .withColumn('rank', rank().over(Window.partitionBy('country').orderBy(desc('max(follower_count)')))) \
                .filter('rank = 1') \
                    .select('country', 'poster_name', 'max(follower_count)') \
                        .withColumnRenamed('max(follower_count)', 'follower_count')

df_most_followers_by_country.show()

df_country_with_most_followed_user = df_most_followers_by_country.groupBy('country') \
    .agg(max('follower_count').alias('follower_count')) \
        .orderBy(desc('follower_count')) \
            .limit(1) \
                .select('country', 'follower_count')

df_country_with_most_followed_user.show()

+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| 9GAG| 3000000|
 Albania| The Minds Journal| 5000000|
 Algeria| Apartment Therapy| 5000000|
 American Samoa| Mamas Uncut| 8000000|
 Andorra|Teachers Pay Teac...| 1000000|
 Angola| Tastemade| 8000000|
 Anguilla| Red Ted Art| 566000|
Antarctica (the t...| Refinery29| 1000000|
 Antigua and Barbuda|Country Living Ma...| 1000000|
 Argentina| Cheezburger| 2000000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| GQ Magazine| 874000|
 Australia| Mujer de 10 | 2000000|
 Austria|Allwomenstalk - g...| 813000|
 Azerbaijan| Style Me Pretty| 6000000|
 Bahamas| Handmade Charlotte| 948000|
 Bahrain| Rainy Day Mum| 337000|
 Bangladesh|Better Homes and ...| 4000000|
 Barbados|The Creativity Ex...| 410000|
 Belarus|Prudent Penny Pin...| 648000|
+--------------------+--------------------+--------------+
only showing top 20 rows

+--------------+--------------+
 country|follower_count|
+--------------+--------------+
American Samoa| 8000000|
+--------------+--------------+

In this query, I've firstly joined the pin and geolocation dataframes called the resulting dataframe 'df_most_followers_by_country'. I've then grouped this dataframe by the 'country' and 'poster_name' columns, as well as added a max() function to get the maximum follower count; next, I've added a Window function that ranks the maximum follower counts corresponding to each poster in descending order, as well as adding a filter so that only the highest ranking poster name for each country based on maximum follower count is shown. I've then selected the 'country', 'poster_name' and 'max(follower_count)' columns, renaming the last column to 'follower_count'.

Using this dataframe, I've then grouped it by 'country', adding a max() function to calculate the most followers for each country, renaming the column 'follower_count'; I've then ordered the dataframe by the 'follower_count' column in descending order, as well as applying a limit of 1 so that only the first entry in the result gets returned, as this will be the country with the user with the most followers. Finally, I've selected the 'country' and 'follower_count' columns and have called the resulting dataframe 'df_country_with_most_followed_user'.

7) Find the most popular category for different age groups

In [ ]:
df_pin_user = join_df(df_pin_cleaned, df_user_cleaned)
df_user_age_group = create_age_groups(df_pin_user)
df_category_count_by_age = df_user_age_group.groupBy("age_group", "category").agg(count("*").alias("category_count")) \
        .groupBy("age_group") \
            .agg(max(struct("category_count", "category")).alias("max_count")) \
                .select("age_group", "max_count.category", "max_count.category_count")

df_category_count_by_age.show()

+---------+--------+--------------+
age_group|category|category_count|
+---------+--------+--------------+
 18-24| tattoos| 446|
 25-35| travel| 231|
 36-50| quotes| 163|
 50+| beauty| 80|
+---------+--------+--------------+

In this query, I've firstly joined the pin and user dataframes, naming the resulting dataframe 'df_pin_user'. I've then applied the 'create_age_groups' function that I created before to this dataframe, calling the resulting dataframe 'df_user_age_group'. Next, I've created a new dataframe called 'df_category_count_by_age', which firstly groups the 'df_user_age_group' dataframe by the 'age_group' and 'category' columns; next, I've added a count() function, which counts the total number of entries, giving it the alias 'category_count'. 

I've then grouped the dataframe by the 'age_group' column, creating a struct column with the alias 'max_count', using the 'category_count' and 'category' columns and applying the max() function to find the highest count for each category, and therefore for each age group as well. Finally, I've selected the 'age_group' column and the 'category' and 'category_count' columns from the 'max_count' struct individually.

8) Find the median follower count for different age groups

In [ ]:
df_median_follower_count_by_age = df_user_age_group.groupBy("age_group") \
    .agg(percentile_approx("follower_count", 0.5, lit(1000000)).alias("median_follower_count")) \
        .select("age_group", "median_follower_count")
df_median_follower_count_by_age.show()

+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 50+| 1000|
 36-50| 6000|
 18-24| 84000|
 25-35| 28000|
+---------+---------------------+

In this query, I've created a dataframe called 'df_median_follower_count_by_age', which takes the 'df_user_age_group' dataframe from the previous query, groups it by the 'age_group' column, then adds a percentile_approx() function with the alias 'median_follower_count', which gives the approximate 50th percentile aka median of the 'follower_count' column for each age group. 'lit(1000000)' ensures that the result is more accurate. Finally, I've selected the 'age_group' and 'median_follower_count' columns.

9) Find how many users have joined each year

In [ ]:
df_user_geo = join_df(df_user_cleaned, df_geo_cleaned)
df_user_year = df_user_geo.select(year("timestamp").alias("post_year"), 'date_joined') \
        .where((df_user_cleaned.date_joined >= '2015-01-01') & (df_user_cleaned.date_joined < '2021-01-01'))

df_user_year.show()

df_users_joined_by_year = df_user_year.groupBy("post_year") \
   .agg(count("*").alias("number_users_joined")) \
        .sort(asc("post_year")) \
            .select("post_year", "number_users_joined")

df_users_joined_by_year.show()

+---------+-------------------+
post_year| date_joined|
+---------+-------------------+
 2022|2015-11-14 05:58:38|
 2022|2015-10-28 03:16:59|
 2022|2015-10-28 03:16:59|
 2022|2015-10-28 03:16:59|
 2022|2015-10-28 03:16:59|
 2021|2016-02-08 06:54:03|
 2020|2017-08-12 08:23:43|
 2021|2016-09-07 04:52:20|
 2021|2016-09-07 04:52:20|
 2021|2016-09-07 04:52:20|
 2021|2016-09-07 04:52:20|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
 2022|2015-10-21 08:27:36|
+---------+-------------------+
only showing top 20 rows

+---------+-------------------+
post_year|number_users_joined|
+---------+-------------------+
 2017| 283|
 2018| 1459|
 2019| 1405|
 2020| 1382|
 2021| 1315|
 2022| 1139|
+---------+-------------------+

For this query, I have firstly joined the user and geolocation dataframes, calling the new dataframe df_user_geo. Using this dataframe, I've then selected the year from the timestamp, calling it 'post_year' and the 'date_joined' column; I've also applied a filter to the dataframe to only display values where the year joined is between 2015 and 2020, with the resulting dataframe being called 'df_user_year'.

Then, using this dataframe, I've grouped it by the 'post_year' column, adding a count() function that counts the total number of entries with the alias 'number_users_joined'; I've sorted the dataframe by the 'post_year' column in ascending order, and finally selected the 'post_year' and 'number_users_joined' columns.

10) Find the median follower count of users based on their joining year

In [ ]:
df_user_geo_pin = join_df(df_user_geo, df_pin_cleaned)
df_follower_joining_year = df_user_geo_pin.select(year("timestamp").alias("post_year"), \
     'date_joined', "follower_count") \
        .where((df_user_cleaned.date_joined >= '2015-01-01') & (df_user_cleaned.date_joined < '2021-01-01'))

df_user_median_follower_count = df_follower_joining_year.groupBy("post_year") \
    .agg(percentile_approx("follower_count", 0.5, lit(1000000)).alias("median_follower_count")) \
        .select("post_year", "median_follower_count")

df_user_median_follower_count.show()

+---------+---------------------+
post_year|median_follower_count|
+---------+---------------------+
 2018| 24000|
 2022| 22000|
 2019| 46000|
 2020| 28000|
 2021| 18000|
 2017| 188000|
+---------+---------------------+

In this query, I've joined the pin dataframe to the user_geo dataframe created in an earlier query, calling the joined dataframe 'df_user_geo_pin'. Then, I've selected the year from the timestamp column, giving it the alias 'post_year', as well as the 'date_joined' and 'follower_count' columns, whilst applying a filter to only include values where the year joined is between 2015 and 2020; the resulting dataframe is called 'df_follower_joining_year'

Next, I've grouped this dataframe by the 'post_year' column, and added a percentile_approx() function like in step 8 above in order to calculate the median, giving it the alias 'median_follower_count'; finally, I've selected the 'post_year' and 'median_follower_count' columns, and called the resulting dataframe 'df_user_median_follower_count'.

11) Find the median follower count of users based on their joining year and age group

In [ ]:
df_user_age_group = create_age_groups(df_user_geo_pin)       
df_user_age_group = df_user_age_group.select("age_group", year("timestamp").alias("post_year"), 'date_joined', "follower_count") \
            .where((df_user_cleaned.date_joined >= '2015-01-01') & (df_user_cleaned.date_joined < '2021-01-01'))
df_median_follower_by_year_joined = df_user_age_group.groupBy("age_group", "post_year") \
    .agg(percentile_approx("follower_count", 0.5, lit(1000000)).alias("median_follower_count")) \
        .select("age_group", "post_year", "median_follower_count")
df_median_follower_by_year_joined.show()

+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 36-50| 2018| 112|
 36-50| 2021| 6000|
 50+| 2018| 9000|
 25-35| 2022| 32000|
 50+| 2022| 3|
 25-35| 2020| 30000|
 18-24| 2019| 51000|
 18-24| 2022| 66000|
 18-24| 2020| 47000|
 25-35| 2019| 51000|
 25-35| 2018| 22000|
 36-50| 2022| 3000|
 25-35| 2021| 27000|
 18-24| 2018| 48000|
 18-24| 2021| 52000|
 50+| 2020| 41|
 36-50| 2020| 9000|
 50+| 2021| 9000|
 50+| 2019| 2000|
 25-35| 2017| 42000|
+---------+---------+---------------------+
only showing top 20 rows

In this query, I've firstly applied the create_age_groups() function to the 'df_user_geo_pin' dataframe. Using this dataframe, I've selected the 'age_group', 'year', 'date_joined' and 'follower_count' columns, giving the 'year' column the alias 'post_year'; I've then added a filter in order to only return values where the joining year is between 2015 and 2020.

Next, I've grouped the dataframe by the 'age_group' and 'post_year' columns, as well as adding a percentile_approx() function to calculate the median like shown above, giving it the alias 'median_follower_count'; finally, I've selected the 'age_group', 'post_year' and 'median_follower_count' columns.

In [ ]:
# Unmount the bucket
dbutils.fs.unmount(MOUNT_NAME)

/mnt/pdp_mount has been unmounted.
Out[247]: True